In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
#from tensorflow.keras.applications.inception_v3 import InceptionV3
 
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [2]:
DATADIR = 'vegetables'
TESTDIR = 'vegetables_test'

IMAGE_SIZE = 224
BATCH_SIZE = 64

traindatagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    zoom_range=[.8, 1],
    rescale=1./255)

testdatagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_generator = traindatagen.flow_from_directory(
    DATADIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

val_generator = testdatagen.flow_from_directory(
    TESTDIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle='False')

Found 7996 images belonging to 19 classes.
Found 1889 images belonging to 19 classes.


In [3]:
for layer in vgg_conv.layers:
    layer.trainable = False

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.AveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(19, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
average_pooling2d (AveragePo (None, 3, 3, 512)         0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2359808   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19)                9

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [6]:
epochs = 5

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)


Epoch 1/5


W0527 00:06:30.662749 12976 deprecation.py:323] From D:\Software\Anaconda\envs\tf_gpu_nightly\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


125/125 [==============================] - 189s 2s/step - loss: 1.9424 - accuracy: 0.3982 - val_loss: 1.1687 - val_accuracy: 0.6384
Epoch 2/5
125/125 [==============================] - 160s 1s/step - loss: 1.2685 - accuracy: 0.5944 - val_loss: 0.9550 - val_accuracy: 0.7088
Epoch 3/5
125/125 [==============================] - 160s 1s/step - loss: 1.1101 - accuracy: 0.6407 - val_loss: 0.9014 - val_accuracy: 0.7136
Epoch 4/5
125/125 [==============================] - 159s 1s/step - loss: 1.0548 - accuracy: 0.6627 - val_loss: 0.8485 - val_accuracy: 0.7300
Epoch 5/5
125/125 [==============================] - 166s 1s/step - loss: 1.0228 - accuracy: 0.6743 - val_loss: 0.7779 - val_accuracy: 0.7597


In [7]:
vgg_conv.trainable = True
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001C735884780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C758354A90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C758354C50> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001C758372F98> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C7583AAFD0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C7583C3D30> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001C7584C77F0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C7584D1B70> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C7584E7E48> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001C7584FA320> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001C7585105C0> False
<tensorflow.python.

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])
filepath="VGG16_model.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
average_pooling2d (AveragePo (None, 3, 3, 512)         0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2359808   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19)                9

In [9]:
history_fine = model.fit(train_generator, 
                         epochs=20,
                         validation_data=val_generator,
                         callbacks=[checkpoint])

Epoch 1/20
124/125 [============================>.] - ETA: 1s - loss: 0.7999 - accuracy: 0.7485
Epoch 00001: val_accuracy improved from -inf to 0.80572, saving model to VGG16_model.best.hdf5
125/125 [==============================] - 176s 1s/step - loss: 0.7972 - accuracy: 0.7494 - val_loss: 0.6113 - val_accuracy: 0.8057
Epoch 2/20
124/125 [============================>.] - ETA: 1s - loss: 0.5379 - accuracy: 0.8282
Epoch 00002: val_accuracy improved from 0.80572 to 0.82901, saving model to VGG16_model.best.hdf5
125/125 [==============================] - 163s 1s/step - loss: 0.5393 - accuracy: 0.8277 - val_loss: 0.5357 - val_accuracy: 0.8290
Epoch 3/20
124/125 [============================>.] - ETA: 1s - loss: 0.4451 - accuracy: 0.8589
Epoch 00003: val_accuracy improved from 0.82901 to 0.85601, saving model to VGG16_model.best.hdf5
125/125 [==============================] - 162s 1s/step - loss: 0.4434 - accuracy: 0.8593 - val_loss: 0.4896 - val_accuracy: 0.8560
Epoch 4/20
124/125 [=====

In [10]:
import numpy as np
val_generator.reset()
Y_pred = model.predict_generator(val_generator)
classes = val_generator.classes[val_generator.index_array]
y_pred = np.argmax(Y_pred, axis=-1)
sum(y_pred==classes)/10000

0.1668

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

img = load_img('vegetables_test/bibimbap/1001113.jpg',target_size=(224, 224))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
prediction = model.predict(x)
maximum = max(prediction[0])
[i for i, j in enumerate(prediction[0]) if j == maximum]
#print(prediction)

[4]

In [29]:
keras_file = "food_vgg16_model.h5"
tf.keras.models.save_model(model, keras_file)